# Data pre-pocessing

In [4]:
# Import the libraries
import numpy as np 
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")
from subprocess import check_output

- There are 887379 lines of loans in the original dataset. To make it easier to work with, we randomly pick 10% from it as a sample set.

In [22]:
# Read the dataset
df = pd.read_csv('loan.csv', low_memory = False)
#Return a random sample of items - 10%
df_sample = df.sample(frac = 0.1)
df_sample_1 = pd.DataFrame(df_sample)

In [128]:
#short-cut to get rid of re-loading the original data -- you may restart here
df_sample=df_sample_1

Then let's analyze the dependent variable - 'loan_status'. 
- There are 10 types of values in this variable, but considering our goal - to tell whether a borrower would default or not - we won't take care of all of them. 
- We only care about those loans either fully paid or paid very late(>121 days), because with the information on hand we can't tell the final result of a current loan or a loan overdue jsut a little bit.
- So we create a 0-1 variable corresponding to these two possible class. Positive class are those loans paid late.

In [129]:
df_sample['loan_status'].value_counts()

Current                                                60108
Fully Paid                                             20938
Charged Off                                             4492
Late (31-120 days)                                      1131
Issued                                                   797
In Grace Period                                          618
Late (16-30 days)                                        228
Does not meet the credit policy. Status:Fully Paid       220
Default                                                  125
Does not meet the credit policy. Status:Charged Off       81
Name: loan_status, dtype: int64

In [130]:
mask = df_sample['loan_status'].isin(['Fully Paid','Charged Off','Default'])
df_sample = df_sample[mask]
df_sample['loan_status'].value_counts()

Fully Paid     20938
Charged Off     4492
Default          125
Name: loan_status, dtype: int64

In [119]:
def CreateTarget(status): 
    if status == 'Fully Paid':
        return 0
    else:
        return 1
    
df_sample['Late_Loan'] = df_sample['loan_status'].map(CreateTarget)
df_sample['Late_Loan'].value_counts()

0    20938
1     4617
Name: Late_Loan, dtype: int64

Then we move on to deal with independent variables:
- Drop out features with more than 25% missing values
- Drop out features that have no or little predictive power and irrelevant to our target
- Replace missing values with 'Unknown' or mean value for numerical features
- Convert ordinal categorical features - 'emp_length' and 'grade' - into numerical
- Performing one-hot encoding on nominal features

In [120]:
# Drop out features with more than 25% missing values
features_missing_series = df_sample.isnull().sum() > len(df_sample)*0.25
features_missing_series = features_missing_series[features_missing_series == True]
features_missing_list =  features_missing_series.index.tolist()
df_sample = df_sample.drop(features_missing_list,axis =1)

# Drop out features that have no or little predictive power and irrelevant to original target
df_sample = df_sample.drop(['id','member_id','loan_status','url','zip_code','policy_code','application_type','issue_d','last_pymnt_d','last_credit_pull_d','verification_status','pymnt_plan','funded_amnt','funded_amnt_inv','sub_grade','out_prncp','out_prncp_inv','total_pymnt_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt','initial_list_status','earliest_cr_line','emp_title','title'],axis =1)

# Replace missing values with 'Unknown' or mean value for numerical features
df_sample['revol_util'].fillna(df_sample['revol_util'].mean(),inplace = True)
df_sample['collections_12_mths_ex_med'].fillna(df_sample['collections_12_mths_ex_med'].mean(),inplace = True)

# Convert ordinal categorical features - 'emp_length' and 'grade' - into numerical 
def EmpLength(length):
    if length == '< 1 year':
        return 0.5
    elif length == '10+ years':
        return 10
    elif length == '1 year':
        return 1
    elif length == '2 years':
        return 2
    elif length == '3 years':
        return 3
    elif length == '4 years':
        return 4
    elif length == '5 years':
        return 5
    elif length == '6 years':
        return 6
    elif length == '7 years':
        return 7
    elif length == '8 years':
        return 8
    elif length == '9 years':
        return 9
    else:
        return 0.5    
df_sample['emp_length_num'] = df_sample['emp_length'].map(EmpLength)
df_sample = df_sample.drop('emp_length',axis =1 )

def GradeInt(x):
    if x == "A":
        return 1
    elif x == "B":
        return 2
    elif x == "C":
        return 3
    elif x == "D":
        return 4
    elif x == "E":
        return 5
    else:
        return 6
df_sample['GradeInt'] = df_sample['grade'].map(GradeInt)
df_sample = df_sample.drop('grade',axis =1 )

# Performing one-hot encoding on nominal features
columns = ['term', 'home_ownership', 'addr_state','purpose']
for col in columns:
    tmp_df = pd.get_dummies(df_sample[col], prefix=col)
    df_sample = pd.concat((df_sample,tmp_df), axis=1)
# Drop out attributes that we've one-hot encoded
df_sample = df_sample.drop(['term', 'home_ownership', 'addr_state','purpose'], axis=1)

In [122]:
df_sample.isnull().sum() # There are no missing values left 

loan_amnt                     0
int_rate                      0
installment                   0
annual_inc                    0
dti                           0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_bal                     0
revol_util                    0
total_acc                     0
collections_12_mths_ex_med    0
acc_now_delinq                0
Late_Loan                     0
emp_length_num                0
GradeInt                      0
term_ 36 months               0
term_ 60 months               0
home_ownership_MORTGAGE       0
home_ownership_NONE           0
home_ownership_OTHER          0
home_ownership_OWN            0
home_ownership_RENT           0
addr_state_AK                 0
addr_state_AL                 0
addr_state_AR                 0
addr_state_AZ                 0
addr_state_CA                 0
addr_state_CO                 0
                             ..
addr_sta

In [123]:
df_sample.dtypes.value_counts()

uint8      71
float64    15
int64       2
dtype: int64

In [124]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25555 entries, 665417 to 218861
Data columns (total 88 columns):
loan_amnt                     25555 non-null float64
int_rate                      25555 non-null float64
installment                   25555 non-null float64
annual_inc                    25555 non-null float64
dti                           25555 non-null float64
delinq_2yrs                   25555 non-null float64
inq_last_6mths                25555 non-null float64
open_acc                      25555 non-null float64
pub_rec                       25555 non-null float64
revol_bal                     25555 non-null float64
revol_util                    25555 non-null float64
total_acc                     25555 non-null float64
collections_12_mths_ex_med    25555 non-null float64
acc_now_delinq                25555 non-null float64
Late_Loan                     25555 non-null int64
emp_length_num                25555 non-null float64
GradeInt                      25555 non

In [82]:
#df_sample.to_csv(
#    "C://Users//tangzhengback//Desktop//df_sample.csv"
#)

#  START HERE

In [126]:
#You may ignore the codes above(don't run). But before you begin, please follow the following steps:
#Step 1: check whether df_sample.csv is in the repository. If not, upload it.
#Step 2：run this cell.

df_sample = pd.read_csv('df_sample.csv', low_memory = False)
df_sample = df_sample.drop([df_sample.columns[0,]],axis=1) 
#Standardize and split the data
y = df_sample.pop('Late_Loan')
column_names = df_sample.columns.values.tolist()
df = df_sample.values
min_max_scaler = preprocessing.MinMaxScaler()
df_scaled = min_max_scaler.fit_transform(df)
df = pd.DataFrame(df_scaled)
c = dict(zip(range(df_sample.columns.size),column_names))
df = df.rename(mapper = c, axis = 'columns')
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=1, stratify=y)

In [127]:
df

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,0.000000,0.201554,0.006938,0.015637,0.065865,0.000000,0.125,0.208333,0.0,0.003519,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.264706,0.444109,0.242959,0.030262,0.499123,0.055556,0.125,0.125000,0.1,0.009201,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.264706,0.564091,0.173368,0.020135,0.453794,0.000000,0.375,0.041667,0.0,0.010561,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.558824,0.583513,0.367455,0.049949,0.349862,0.000000,0.375,0.250000,0.0,0.017420,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.488971,0.570997,0.319705,0.020137,0.375657,0.000000,0.000,0.187500,0.0,0.022874,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.044118,0.062149,0.040142,0.010241,0.046832,0.000000,0.000,0.125000,0.0,0.001028,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.264706,0.359517,0.235843,0.020137,0.104433,0.000000,0.125,0.062500,0.0,0.006040,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.035294,0.318084,0.019908,0.015640,0.595292,0.000000,0.000,0.166667,0.0,0.004577,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.558824,0.503669,0.352500,0.037012,0.453794,0.000000,0.250,0.166667,0.0,0.014581,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.264706,0.293483,0.230369,0.031387,0.253944,0.000000,0.000,0.166667,0.0,0.028041,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
#########################################
#    This is just a test, ignore it    #
#######################################


# Fitting decision tree to the training data: Model_1
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train,y_train)

# Predicting the results
y_pred = classifier.predict(X_test)

# Creating the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
accuracy = (cm[0,0]+cm[1,1])/len(y_test)
print("Accuracy_DecisionTree:",accuracy*100,'%')
print(cm)


Accuracy_DecisionTree: 72.03599843485065 %
[[5167 1115]
 [1029  356]]
